In [1]:
from gbm.data import Trigdat

%matplotlib inline
import matplotlib.pyplot as plt
from gbm.plot import Lightcurve

from gbm import test_data_dir
from gbm.data import TTE
from gbm.plot import Spectrum
import matplotlib.pyplot as plt
from gbm.data.primitives import TimeBins, EnergyBins
from gbm.data.primitives import TimeEnergyBins

from gbm.binning.unbinned import bin_by_time
import numpy as np

from scipy.signal import savgol_filter
from scipy.ndimage import median_filter
import pywt
#from pyrobust import pca
from scipy.ndimage import binary_erosion, binary_dilation

from scipy.optimize import curve_fit

from gbm.finder import TriggerCatalog, TriggerFtp
import os







/Users/vytis/anaconda3/lib/python3.10/site-packages/gbm/plot/lal_post_subs.py:184: UserWarning: Basemap not installed. Some functionality not available.
  warnings.warn('Basemap not installed. Some functionality not available.')


In [4]:
def polynomial(x, *coeffs):
    return np.polyval(coeffs, x)

def subtract_background_iteratively(bin_times, bin_counts, degree=2, iterations=2):
    corrected_counts = bin_counts.copy()
    for iteration in range(iterations):
        for i in range(corrected_counts.shape[0]):
            background_mask = np.logical_or(bin_times < 0, bin_times > 20)
            coeffs, _ = curve_fit(polynomial, bin_times[background_mask], corrected_counts[i, background_mask], p0=[1] * (degree + 1))
            background = polynomial(bin_times, *coeffs)
            corrected_counts[i, :] -= background
            corrected_counts[i, :] = np.clip(corrected_counts[i, :], 0, None)

    noise_level = np.std(corrected_counts[:, background_mask], axis=1)
    snr_threshold = 2
    for i in range(corrected_counts.shape[0]):
        signal = corrected_counts[i, :]
        noise = noise_level[i]
        corrected_counts[i, :] = np.where(signal > snr_threshold * noise, signal, 0)

    return corrected_counts

def process_tte_data(detector_name, bin_number, time_range, smearing_factor=0.1):
    try:
        tte = TTE.open(f'glg_tte_{detector_name}_{bin_number}_v00.fit')

        time_sliced_tte = tte.slice_time(time_range)

        eventlist = time_sliced_tte.data

        bin_width = 0.2
        bins = eventlist.bin(bin_by_time, bin_width)

        bin_times = bins.time_centroids
        bin_counts = bins.counts
        energy = bins.energy_centroids

        # Smear the bin counts
        smeared_counts = np.copy(bin_counts)
        for i in range(1, len(bin_counts) - 1):
            smeared_counts[i] = (smearing_factor * bin_counts[i - 1] +
                                 (1 - 2 * smearing_factor) * bin_counts[i] +
                                 smearing_factor * bin_counts[i + 1])

        return bin_times, smeared_counts, energy
    except AttributeError as e:
        print(f"Error processing TTE data for {detector_name}: {e}")
        return None, None, None  # Return None values to indicate failure
    
time_ranges = [(-15, 15), (-50, 50), (-150, 150)]    

# Step 1: Choose GRBs of Interest
trigcat = TriggerCatalog()
sliced_trigcat = trigcat.slice('end_time')

# Step 2: Initialize Trigger Finder
for trigger_info in sliced_trigcat.get_table(columns=('trigger_name', 'trigger_time')).tolist()[:5]:
    trigger_name, trigger_time = trigger_info
    print(f"Downloading TTE files for GRB {trigger_name} ({trigger_time})")
    
    # Remove "bn" from trigger_name
    modified_trigger_name = trigger_name.replace("bn", "")

    # Initialize Trigger Finder
    trig_finder = TriggerFtp(modified_trigger_name)

    # Download trigdat file to the current directory
    try:
        trig_finder.get_trigdat('./')
    except AttributeError as e:
        print(f"AttributeError occurred while downloading trigdat file for GRB {trigger_name}: {e}")
        continue
      
    # Try opening the file with v01, if not found, try v00 and v02
    version_suffixes = ['v01', 'v00', 'v02']
    for suffix in version_suffixes:
        filename = f'glg_trigdat_all_{trigger_name}_{suffix}.fit'
        try:
            trigdat = Trigdat.open(filename)
            trig_dets = trigdat.triggered_detectors
            file_found = True
            break  # If the file is successfully opened, exit the loop
        except OSError as e:
            if "does not exist" in str(e):
                print(f"File {filename} does not exist. Trying next version.")
            else:
                raise  # Re-raise the exception if it's not related to file existence

    if not file_found:
        print("None of the files with the specified versions were found.")
    
    # Download TTE files to the current directory
    trig_finder.get_tte('./')

    print(f"Download complete for GRB {trigger_name}\n")

    for time_range in time_ranges:
        
        time_range_dir = f'/Users/vytis/Desktop/GRB Images/TimeRange_{time_range[0]}_{time_range[1]}'
        os.makedirs(time_range_dir, exist_ok=True)
        
        # Process the downloaded TTE data
        bin_number = modified_trigger_name
        if not bin_number.startswith('bn'):
            bin_number = 'bn' + bin_number
        length = "Long"
        grb_name = bin_number.replace("bn", "")

        # NaI Detectors
        na_detectors = trig_dets
        na_counts_list = []
        for detector in na_detectors:
            bin_times, bin_counts, _ = process_tte_data(detector, bin_number, time_range)

            # Remove the bottom 90% of counts for each column
            percentile_90 = np.percentile(bin_counts, 90, axis=0)
            for i in range(bin_counts.shape[1]):
                bin_counts[:, i] = np.where(bin_counts[:, i] > percentile_90[i], bin_counts[:, i], 0)

            #bin_counts_nai_corrected = subtract_background(bin_times_nai, bin_counts_nai.T).T
            bin_counts_polynomial_filtering = subtract_background_iteratively(bin_times, bin_counts.T).T

            # Set the last two rows to 0
            bin_counts_polynomial_filtering[:, -3:] = 0
            
            # Normalize the filtered counts
            bin_counts_normalized = bin_counts_polynomial_filtering / np.max(bin_counts_polynomial_filtering)

            # Apply median filter
            filtered_counts = median_filter(bin_counts_normalized, size=(2, 2))

            na_counts_list.append(filtered_counts)

        try:
            # Combine processed counts from all NaI detectors
            bin_counts_nai = np.sum(na_counts_list, axis=0)
            filtered_counts_log_nai = np.log1p(bin_counts_nai)
        except ValueError as e:
            print(f"ValueError occurred while combining detector counts: {e}")
            # Handle the error case by skipping the current loop iteration
            continue

        bin_times_list, _, energy_list = zip(*[process_tte_data(detector, bin_number, time_range) for detector in na_detectors])

        # Combine counts from all NaI detectors
        bin_times_nai = bin_times_list[0]
        energy_nai = energy_list[0]

        # Bismuth Detectors
        bgo_detectors = ['b0', 'b1']
        bgo_counts_list = []
        for detector in bgo_detectors:
            bin_times, bin_counts, _ = process_tte_data(detector, bin_number, time_range)

            try:
                percentile_90 = np.percentile(bin_counts, 90, axis=0)
                for i in range(bin_counts.shape[1]):
                    bin_counts[:, i] = np.where(bin_counts[:, i] > percentile_90[i], bin_counts[:, i], 0)
            except np.AxisError as e:
                print(f"AxisError occurred while processing percentile for detector {detector}: {e}")
                continue

            # Normalize the filtered counts
            bin_counts_normalized = bin_counts / np.max(bin_counts)

            # Apply median filter
            filtered_counts = median_filter(bin_counts_normalized, size=(2, 2))

            bgo_counts_list.append(filtered_counts)

        # Combine processed counts from all NaI detectors
        bin_counts_bgo = np.sum(bgo_counts_list, axis=0)
        filtered_counts_log_bgo = np.log1p(bin_counts_bgo)

        bin_times_bgo, bin_counts_bgo, energy_bgo = zip(*[process_tte_data(detector, bin_number, time_range) for detector in bgo_detectors])

        # Combine counts from all BGO detectors
        bin_times_bgo_combined = bin_times_bgo[0]
        energy_bgo_combined = energy_bgo[0]

        # Create the figure
        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 8))

        # Plot for NaI Detectors
        im1 = ax1.imshow(filtered_counts_log_bgo.T, aspect='auto', extent=[time_range[0], time_range[1], energy_bgo_combined.min(), energy_bgo_combined.max()], cmap='viridis', origin='lower')
        ax1.set_ylabel('Energy (keV)')
        ax1.set_title(f'GRB {grb_name} ({length}) Count Heat Map for Time Range {time_range[0]} to {time_range[1]}')
        fig.colorbar(im1, ax=ax1, label='Log(Counts + 1)')
        
        # Plot for BGO Detectors
        im2 = ax2.imshow(filtered_counts_log_nai.T, aspect='auto', extent=[time_range[0], time_range[1], energy_nai.min(), energy_nai.max()], cmap='viridis', origin='lower')
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('Energy (keV)')
        ax2.set_title('')
        fig.colorbar(im2, ax=ax2, label='Log(Counts + 1)')

        plt.subplots_adjust(hspace=0.01)
        plt.savefig(os.path.join(time_range_dir, f'GRB_{grb_name}_TimeRange_{time_range[0]}_{time_range[1]}.png'))
        plt.close()
        print(f'Image for time range {time_range[0]} to {time_range[1]} successfully created.')

Finished in 7 s
Connection appears to have failed.  Attempting to reconnect...
Reconnected.
glg_trigdat_all_bn120403857_v01.fit [==============================] 100.00%
glg_tte_b0_bn120403857_v00.fit [==============================] 100.00%
glg_tte_b1_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n0_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n1_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n2_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n3_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n4_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n5_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n6_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n7_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n8_bn120403857_v00.fit [==============================] 100.00%
glg_tte_n9_bn120403857_v00.fit [=======

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



glg_tte_n0_bn140630748_v00.fit [==============================] 100.00%
glg_tte_n1_bn140630748_v00.fit [==============================] 100.00%
glg_tte_n2_bn140630748_v00.fit [==============================] 100.00%
glg_tte_n3_bn140630748_v00.fit [==============================] 100.00%
glg_tte_n4_bn140630748_v00.fit [==============================] 100.00%
glg_tte_n5_bn140630748_v00.fit [==============================] 100.00%
glg_tte_n6_bn140630748_v00.fit [==============================] 100.00%
glg_tte_n7_bn140630748_v00.fit [==============================] 100.00%
glg_tte_n8_bn140630748_v00.fit [==============================] 100.00%
glg_tte_n9_bn140630748_v00.fit [==============================] 100.00%
glg_tte_na_bn140630748_v00.fit [==============================] 100.00%
glg_tte_nb_bn140630748_v00.fit [=================             ] 59.90%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Image for time range -15 to 15 successfully created.
Image for time range -50 to 50 successfully created.
Image for time range -150 to 150 successfully created.
Connection appears to have failed.  Attempting to reconnect...
Reconnected.
glg_trigdat_all_bn230524357_v02.fit [==============================] 100.00%
File glg_trigdat_all_bn230524357_v01.fit does not exist. Trying next version.
File glg_trigdat_all_bn230524357_v00.fit does not exist. Trying next version.
glg_tte_b0_bn230524357_v00.fit [==============================] 100.00%
glg_tte_b1_bn230524357_v00.fit [==============================] 100.00%
glg_tte_n0_bn230524357_v00.fit [==============================] 100.00%
glg_tte_n1_bn230524357_v00.fit [==============================] 100.00%
glg_tte_n2_bn230524357_v00.fit [==============================] 100.00%
glg_tte_n3_bn230524357_v00.fit [==============================] 100.00%
glg_tte_n4_bn230524357_v00.fit [==============================] 100.00%
glg_tte_n5_bn230524357_v00

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

# Image dimensions
img_width, img_height = 150, 150
train_data_dir = 'path_to_training_data'
validation_data_dir = 'path_to_validation_data'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# Model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Rescaling for validation
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Training the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

# Saving the model
model.save('model.h5')